In [ ]:
# !pip install Pillow
# !pip install tf-nightly
# !pip install pandas
# !pip install ibm_boto3
# !pip install botocore
# !pip install ibm-cos-sdk
# !pip install imageio
# !pip install seaborn
# !pip install plotly
# !pip install tensorflow_hub
# !pip install edge_ai_model_mgmt
# !pip install tensorflow-model-optimization
# !pip install edge_ai_model_mgmt
# !pip install edgeaimodelmgmt


In [2]:
import numpy as np # linear algebra
import pandas as pd
# import cv2
import os, types
import math
import imageio
import seaborn as sns
from glob import glob
import pathlib
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import Image
import PIL
from PIL import Image
import time


from botocore.client import Config
import ibm_boto3
import zipfile
import shutil

import requests
import pickle
import json
import warnings
warnings.filterwarnings('ignore')

# from sklearn.model_selection import train_test_split

In [3]:
import seaborn as sns
import plotly.express as px
from IPython.display import SVG
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

%matplotlib inline

In [4]:
import tempfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.applications import VGG16,inception_v3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
from tensorflow.keras.layers import Input, Lambda,Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                                    Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
                                    LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape,\
                                    Conv2DTranspose, LeakyReLU, Conv1D, AveragePooling1D, MaxPooling1D

from tensorflow.keras.models import load_model
import tensorflow_hub as hub
# import tensorflow_model_optimization as tfmot
# from edgeaimodelmgmt.ode.tf_pruner import TfPruner
# from edgeaimodelmgmt.ode.tf_quantizer import TfQuantizer

In [5]:
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.4.1
Hub version: 0.12.0
GPU is NOT AVAILABLE


2022-03-31 18:13:28.516821: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [6]:
bucket_name = 'gurvsin3-visualrecognition'
object_key = 'fire_dataset.zip'

DATA_DIR = os.path.join(os.path.abspath(os.path.pardir),'data')
RESULTS_DIR = os.path.join('../results/fire_detection', 'tfhub')
dataZip = os.path.join(DATA_DIR, object_key)
dataset_dir = pathlib.Path(DATA_DIR+'/fire_dataset')

batch_size = 32
IMAGE_SIZE = (224, 224)


In [7]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': '816HFeHbtI_WiIlkFTaAO61vld7bM7nDB08U7MpaVHNm',
    'COS_SERVICE_CRN': 'crn:v1:bluemix:public:iam-identity::a/2c303e22de2b34062121bab41b176b1b::serviceid:ServiceId-3152b935-e74b-4e7d-8460-6e9d56d166e8',
    'ENDPOINT': 'https://s3.jp-tok.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/identity/token',
    'BUCKET': 'myml-donotdelete-pr-zhsoop3fasxh7h',
    'FILE': 'fire_dataset.zip'
}


In [8]:
def loadDataFromCOS(bucket_name, object_key):
    
    def __iter__(self): return 0

    cos_cli = ibm_boto3.client(service_name='s3',
        ibm_api_key_id=credentials_1["IBM_API_KEY_ID"],
        ibm_service_instance_id=credentials_1["COS_SERVICE_CRN"],
        ibm_auth_endpoint=credentials_1["IBM_AUTH_ENDPOINT"],
        config=Config(signature_version="oauth"),
        endpoint_url=credentials_1["ENDPOINT"]
    )

    # Your data file was loaded into a botocore.response.StreamingBody object.
    # Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
    # ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
    # pandas documentation: http://pandas.pydata.org/
    streaming_body_1 = cos_cli.get_object(Bucket=bucket_name, Key=object_key)['Body']
    # add missing __iter__ method, so pandas accepts body as file-like object
    if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 
        
    f = open(object_key, 'wb')
    f.write(streaming_body_1.read())
    

In [9]:
def unzip_dataset(file_path):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)

In [10]:
def load_dataset():
    if os.path.exists(object_key):
        print('Data ZIP file exists')
        if(os.path.isdir(DATA_DIR)):
            print('Dataset unziped Exists')
        else:
            print('Unzip required..')
            unzip_dataset(object_key)
    else:
        print('No Data')
        loadDataFromCOS(bucket_name, object_key)
        unzip_dataset(object_key)

load_dataset()


Data ZIP file exists
Dataset unziped Exists


In [ ]:
# UNCOMMENT ONLY TO REMOVE FILES
# shutil.rmtree(data_dir)

In [ ]:
# data_dir = pathlib.Path(data_dir_path+'/fire_dataset')
jpg_image_count = len(list(dataset_dir.glob('*/*.jpg')))
png_image_count = len(list(dataset_dir.glob('*/*.png')))
print(jpg_image_count, png_image_count)

In [ ]:
fire_images = list(dataset_dir.glob('fire/*'))
PIL.Image.open(str(fire_images[0]))

In [ ]:
non_fire_images = list(dataset_dir.glob('non_fire/*'))
PIL.Image.open(str(non_fire_images[0]))

In [11]:
model_name = "efficientnetv2-b0-21k" # @param ['efficientnetv2-s', 'efficientnetv2-m', 'efficientnetv2-l', 'efficientnetv2-s-21k', 'efficientnetv2-m-21k', 'efficientnetv2-l-21k', 'efficientnetv2-xl-21k', 'efficientnetv2-b0-21k', 'efficientnetv2-b1-21k', 'efficientnetv2-b2-21k', 'efficientnetv2-b3-21k', 'efficientnetv2-s-21k-ft1k', 'efficientnetv2-m-21k-ft1k', 'efficientnetv2-l-21k-ft1k', 'efficientnetv2-xl-21k-ft1k', 'efficientnetv2-b0-21k-ft1k', 'efficientnetv2-b1-21k-ft1k', 'efficientnetv2-b2-21k-ft1k', 'efficientnetv2-b3-21k-ft1k', 'efficientnetv2-b0', 'efficientnetv2-b1', 'efficientnetv2-b2', 'efficientnetv2-b3', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'bit_s-r50x1', 'inception_v3', 'inception_resnet_v2', 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152', 'nasnet_large', 'nasnet_mobile', 'pnasnet_large', 'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

model_handle_map = {
  "efficientnetv2-s": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2",
  "efficientnetv2-m": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_m/feature_vector/2",
  "efficientnetv2-l": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/feature_vector/2",
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/feature_vector/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/feature_vector/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/feature_vector/2",
  "efficientnetv2-s-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/feature_vector/2",
  "efficientnetv2-m-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/feature_vector/2",
  "efficientnetv2-l-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_l/feature_vector/2",
  "efficientnetv2-xl-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2",
  "efficientnetv2-b0-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/feature_vector/2",
  "efficientnetv2-b1-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b1/feature_vector/2",
  "efficientnetv2-b2-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b2/feature_vector/2",
  "efficientnetv2-b3-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2",
  "efficientnetv2-b0": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",
  "efficientnetv2-b1": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/feature_vector/2",
  "efficientnetv2-b2": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/feature_vector/2",
  "efficientnetv2-b3": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/feature_vector/2",
  "efficientnet_b0": "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
  "efficientnet_b1": "https://tfhub.dev/tensorflow/efficientnet/b1/feature-vector/1",
  "efficientnet_b2": "https://tfhub.dev/tensorflow/efficientnet/b2/feature-vector/1",
  "efficientnet_b3": "https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1",
  "efficientnet_b4": "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1",
  "efficientnet_b5": "https://tfhub.dev/tensorflow/efficientnet/b5/feature-vector/1",
  "efficientnet_b6": "https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1",
  "efficientnet_b7": "https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1",
  "bit_s-r50x1": "https://tfhub.dev/google/bit/s-r50x1/1",
  "inception_v3": "https://tfhub.dev/google/imagenet/inception_v3/feature-vector/4",
  "inception_resnet_v2": "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature-vector/4",
  "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/feature-vector/4",
  "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/feature-vector/4",
  "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/feature-vector/4",
  "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4",
  "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/feature-vector/4",
  "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/feature-vector/4",
  "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/4",
  "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/4",
  "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/feature_vector/4",
  "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
  "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
  "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
  "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
  "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
  "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
  "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
}

model_image_size_map = {
  "efficientnetv2-s": 384,
  "efficientnetv2-m": 480,
  "efficientnetv2-l": 480,
  "efficientnetv2-b0": 224,
  "efficientnetv2-b1": 240,
  "efficientnetv2-b2": 260,
  "efficientnetv2-b3": 300,
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "efficientnetv2-s-21k-ft1k": 384,
  "efficientnetv2-m-21k-ft1k": 480,
  "efficientnetv2-l-21k-ft1k": 480,
  "efficientnetv2-xl-21k-ft1k": 512,
  "efficientnetv2-b0-21k-ft1k": 224,
  "efficientnetv2-b1-21k-ft1k": 240,
  "efficientnetv2-b2-21k-ft1k": 260,
  "efficientnetv2-b3-21k-ft1k": 300, 
  "efficientnet_b0": 224,
  "efficientnet_b1": 240,
  "efficientnet_b2": 260,
  "efficientnet_b3": 300,
  "efficientnet_b4": 380,
  "efficientnet_b5": 456,
  "efficientnet_b6": 528,
  "efficientnet_b7": 600,
  "inception_v3": 299,
  "inception_resnet_v2": 299,
  "nasnet_large": 331,
  "pnasnet_large": 331,
}

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name, 224)

print(f"Selected model: {model_name} : {model_handle}")

IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 32

Selected model: efficientnetv2-b0-21k : https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2
Input size (224, 224)


In [12]:
def build_dataset(subset):
    return tf.keras.preprocessing.image_dataset_from_directory(
        dataset_dir,
        validation_split=.20,
        subset=subset,
        label_mode="categorical",
        # Seed needs to provided when using validation_split and shuffle = True.
        # A fixed seed is used so that the validation set is stable across runs.
        seed=123,
        image_size=IMAGE_SIZE,
        batch_size=1)

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])
do_data_augmentation = False
if do_data_augmentation:
    preprocessing_model.add(
      tf.keras.layers.experimental.preprocessing.RandomRotation(40))
    preprocessing_model.add(
      tf.keras.layers.experimental.preprocessing.RandomTranslation(0, 0.2))
    preprocessing_model.add(
      tf.keras.layers.experimental.preprocessing.RandomTranslation(0.2, 0))
  # Like the old tf.keras.preprocessing.image.ImageDataGenerator(),
  # image sizes are fixed when reading, and then a random zoom is applied.
  # If all training inputs are larger than image_size, one could also use
  # RandomCrop with a batch size of 1 and rebatch later.
    preprocessing_model.add(
      tf.keras.layers.experimental.preprocessing.RandomZoom(0.2, 0.2))
    preprocessing_model.add(
      tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal"))
    
train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))


Found 1650 files belonging to 2 classes.
Using 1320 files for training.
Found 1650 files belonging to 2 classes.
Using 330 files for validation.


2022-03-31 18:13:43.876483: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# datagen = ImageDataGenerator(validation_split=0.2, 
#                              rescale=1./255,
#                              preprocessing_function=preprocess_input)
# train = datagen.flow_from_directory(data_dir,
#                                               target_size=(img_width, img_height), color_mode='rgb',
#                                               class_mode='binary',shuffle=True, batch_size=32,subset="training" )
# valid = datagen.flow_from_directory(data_dir,
#                                               target_size=(img_width, img_height), color_mode='rgb',
#                                               class_mode='binary',shuffle=True, batch_size=8,subset="validation")

In [37]:
def get_model():
#     model = Sequential([
#       layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
#       layers.Conv2D(16, 3, padding='same', activation='relu'),
#       layers.MaxPooling2D(),
#       layers.Conv2D(32, 3, padding='same', activation='relu'),
#       layers.MaxPooling2D(),
#       layers.Conv2D(64, 3, padding='same', activation='relu'),
#       layers.MaxPooling2D(),
#       layers.Flatten(),
#       layers.Dense(128, activation='relu'),
#       layers.Dense(num_classes)
#     ])
#     model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])


    do_fine_tuning = False
    print("Building model with", model_handle)
    INPUT_SHAPE = (None, 224, 224, 3)
    model = tf.keras.Sequential([
        # Explicitly define the input shape so the model can be properly
        # loaded by the TFLiteConverter
        layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
        hub.KerasLayer(model_handle, trainable=do_fine_tuning, input_shape=INPUT_SHAPE),
        layers.Dropout(rate=0.2),
        layers.Dense(len(class_names),
                              kernel_regularizer=tf.keras.regularizers.l2(0.0001))
    ])
    model.build((None,)+IMAGE_SIZE+(3,))
    
    model.summary()

    return model

In [38]:
model = get_model()
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

# model.compile(
#   optimizer=tf.keras.optimizers.Adam(lr=0.005),
#   loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
#   metrics=['accuracy'])

Building model with https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   (None, 1280)              5919312   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 2562      
Total params: 5,921,874
Trainable params: 2,562
Non-trainable params: 5,919,312
_________________________________________________________________


In [39]:
Callback_Stop_Early = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=3,mode='auto')

steps_per_epoch = train_size // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE
history = model.fit(
    train_ds,
    epochs=15, steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    callbacks=[Callback_Stop_Early],
    validation_steps=validation_steps).history

Epoch 1/15
41/41 [==============================] - 84s 2s/step - loss: 0.8882 - accuracy: 0.8485 - val_loss: 0.4192 - val_accuracy: 0.9688
Epoch 2/15
41/41 [==============================] - 76s 2s/step - loss: 0.4142 - accuracy: 0.9465 - val_loss: 0.3214 - val_accuracy: 0.9625
Epoch 3/15
41/41 [==============================] - 76s 2s/step - loss: 0.3821 - accuracy: 0.9450 - val_loss: 0.3780 - val_accuracy: 0.9375
Epoch 4/15
41/41 [==============================] - 76s 2s/step - loss: 0.3530 - accuracy: 0.9517 - val_loss: 0.4167 - val_accuracy: 0.9625
Epoch 5/15
41/41 [==============================] - 78s 2s/step - loss: 0.3941 - accuracy: 0.9587 - val_loss: 0.3663 - val_accuracy: 0.9531


In [ ]:
def plot_graph(history):
    print(history.keys())
    sns.set()
    # summarize history for accuracy
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
plot_graph(history)

In [13]:
model_save_path = os.path.join(RESULTS_DIR, 'fire_tfhub.h5')
print(model_save_path)

../results/fire_detection/tfhub/fire_tfhub.h5


In [ ]:
model.save(model_save_path)

In [25]:
# from tensorflow.keras.models import load_model
model = load_model(model_save_path, custom_objects={'KerasLayer':hub.KerasLayer})
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              5919312   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2562      
Total params: 5,921,874
Trainable params: 2,562
Non-trainable params: 5,919,312
_________________________________________________________________


In [ ]:
# tf.saved_model.save(model, model_save_path)

In [ ]:
def convert_to_lite(model):
    optimize_lite_model = False 
    num_calibration_examples = 60 
    representative_dataset = None
    if optimize_lite_model and num_calibration_examples:
      # Use a bounded number of training examples without labels for calibration.
      # TFLiteConverter expects a list of input tensors, each with batch size 1.
        representative_dataset = lambda: itertools.islice(
            ([image[None, ...]] for batch, _ in train_ds for image in batch),
            num_calibration_examples)

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimize_lite_model:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        if representative_dataset:  # This is optional, see above.
            converter.representative_dataset = representative_dataset
    lite_model_content = converter.convert()

    with open(RESULTS_DIR+'/model.tflite', "wb") as f:
        f.write(lite_model_content)
    print("Wrote %sTFLite model of %d bytes." %
          ("optimized " if optimize_lite_model else "", len(lite_model_content)))

convert_to_lite(model)

In [ ]:
x, y = next(iter(val_ds))
image = x[0, :, :, :]
true_index = np.argmax(y[0])
plt.imshow(image)
plt.axis('off')
plt.show()

print(np.expand_dims(image, axis=0).shape)

# Expand the validation image to (1, 224, 224, 3) before predicting the label
prediction_scores = model.predict(np.expand_dims(image, axis=0))
print('prediction_scores: ', prediction_scores)
predicted_index = np.argmax(prediction_scores)
print("True label: " + class_names[true_index])
print("Predicted label: " + class_names[predicted_index])

# loss, acc = model.evaluate(val_ds, verbose=2)

In [ ]:
!rm -rf ../data/datasets/test_img
class_names = ['fire', 'non_fire']

# test_img_url = 'https://climate.esa.int/media/images/Fire-CCI-banner_1.2e16d0ba.fill-600x314-c100.format-jpeg.jpg'
# test_img_url = 'https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/small-kitchen-1572367025.png'
# test_img_url = 'https://images.indianexpress.com/2021/03/Pune-fire-1.jpg'
# test_img_url = 'https://i.insider.com/56843d15dd0895dc648b47fd?width=1000&format=jpeg&auto=webp'
# test_img_url = 'https://charlestonfire.org/images/public-news/large/news-1501509679.jpg' 
# test_img_url = 'https://cf.ltkcdn.net/safety/images/std/116578-232x317r1-Kitchenfireprevention.jpg' # No_Fire
# test_img_url ='https://www.kutchina.com/wp-content/uploads/2020/07/straight-line-2-min-600x350.jpg' # No_Fire
# test_img_url = 'https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/kitchen-ideas-wall-display-1603743563.jpg'
# test_img_url = 'https://media.springernature.com/lw725/springer-cms/rest/v1/content/19831396/data/v1' # False Negative
# test_img_url = 'https://media-exp1.licdn.com/dms/image/C4D1BAQGrF7AoQDBPXQ/company-background_10000/0/1631550860606?e=2147483647&v=beta&t=0MXoiDfXevwDLrZ8ZHFEXxlcS0T_iyMShpgHlunLMQo'
# test_img_url = 'https://www.thespruce.com/thmb/XMuqYgCjvYclSkDrm7ALrwnoXw8=/1777x1333/smart/filters:no_upscale()/fire-pit-backyard-ideas-4177680-hero-4b2d98b9204b4cd4bdb534a5a9b106f4.jpg' # False Negative

test_img_url = 'https://cdn.britannica.com/49/161649-050-3F458ECF/Bernese-mountain-dog-grass.jpg?q=60'

test_img_path = tf.keras.utils.get_file('test_img', origin=test_img_url, cache_dir='../data')
# test_img_path = '/Users/gurvindersingh/Downloads/fire.jpeg'

# test_img_path = '../data/fire_dataset/non_fire/10.jpg'

print(test_img_path)

img = keras.preprocessing.image.load_img(
    test_img_path, target_size=IMAGE_SIZE
)

plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# image = Image.open(test_img_path).convert('RGB').resize(IMAGE_SIZE, Image.ANTIALIAS)
# img_array = np.float32(image) / 255
img_array = keras.preprocessing.image.img_to_array(img) / 255
# img_array = (np.float32(image) - 123) / 123
# img_array = tf.expand_dims(img_array, 0) # Create a batch
# img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(np.expand_dims(img_array, axis=0))
print("predictions: ", predictions)
score = tf.nn.softmax(predictions[0])
predicted_index = np.argmax(predictions)
score = tf.nn.softmax(predictions[0])

# print('score: ', score)
print('Class: ', np.argmax(score))
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
# Load the TFLite model and allocate tensors.

interpreter = tf.lite.Interpreter(model_path=RESULTS_DIR+'/model.tflite')
interpreter.allocate_tensors()

_, height, width, _ = interpreter.get_input_details()[0]['shape']
print('width: ', width, ', height: ', height)
input_index = interpreter.get_input_details()[0]["index"]
output_details = interpreter.get_output_details()[0]
output_index = output_details["index"]

In [ ]:
def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

def classify_image(interpreter, image, top_k=1):
    """Returns a sorted array of classification results."""
#     set_input_tensor(interpreter, image)
    interpreter.set_tensor(input_index, image)
    interpreter.invoke()
    
#     output = np.squeeze(interpreter.get_tensor(output_index['index']))
    output = interpreter.tensor(output_index)
#     digit = np.argmax(output()[0])
#     confidence = np.max(output()[0])
    score = (np.exp(output()).T / np.exp(output()).sum(axis=-1)).T
    print('score', score)
    # If the model is quantized (uint8 data), then dequantize the results
    if output_details['dtype'] == np.uint8:
        scale, zero_point = output_details['quantization']
        output = scale * (output - zero_point)
        print('quantized model output: >> ', output)

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

In [ ]:
image = Image.open(test_img_path).convert('RGB').resize(IMAGE_SIZE, Image.ANTIALIAS)
img_array = np.float32(image) / 255

img_array = np.expand_dims(img_array, axis=0)

print(img_array.shape)

result = classify_image(interpreter, img_array)


In [16]:
import tensorflow_model_optimization as tfmot
from edgeaimodelmgmt.ode.tf_pruner import TfPruner
from edgeaimodelmgmt.ode.tf_quantizer import TfQuantizer
from modelmgmt.fingerprinting.model_fingerprint import ModelFingerprint
from modelmgmt.fingerprinting.fingerprint import Fingerprint
from modelmgmt.models.keras_model import KerasModel

In [17]:
x_train = []
y_train = []
x_test = []
y_test = []

for i, (image, label) in enumerate(train_ds.unbatch().take(1320)):
    x_train.append(image.numpy())
    y_train.append(label.numpy())
    
for i, (image, label) in enumerate(val_ds.unbatch().take(330)):
    x_test.append(image.numpy())
    y_test.append(label.numpy())

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)


2022-03-31 18:15:03.148133: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


In [20]:
print(x_train.shape)

(1320, 224, 224, 3)


In [29]:
# custom_objects={'KerasLayer':hub.KerasLayer}

# with tfmot.quantization.keras.quantize_scope({'KerasLayer':hub.KerasLayer}):
#     loaded_model = load_model(model_save_path)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              5919312   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2562      
Total params: 5,921,874
Trainable params: 2,562
Non-trainable params: 5,919,312
_________________________________________________________________


In [28]:
q = TfQuantizer(model)

with tfmot.quantization.keras.quantize_scope({'KerasLayer':hub.KerasLayer}):
    q_aware_model, err = q.quantize(np.int8, 8, x_train, y_train, x_test, y_test, 
                     include_biases=False, optimizer=tf.keras.optimizers.SGD)


# Evaluate the model
if not err:
    loss, acc = q_aware_model.evaluate(x_test, y_test, verbose=2)
    print("Quant trained model, accuracy: {:5.2f}%".format(100*acc))
else:
    print("Could not quantize the model...")
#     exit() # terminate execution as we cannot continue with the script...

ERROR:edgeaimodelmgmt.ode.tf_quantizer:Could not quantize the model due to: Cannot assign to variable efficientnetv2-b0/blocks_0/conv2d/kernel:0 due to variable shape (3, 3, 32, 16) and value shape (1, 1, 192, 1280) are incompatible


Could not quantize the model...


In [ ]:
headers = {'X-IBM-Client-Id': '4ada90b8-88ec-4ef5-88a5-2c15df158069',
           'X-IBM-Client-Secret':'V0hT7sL5yN5cK5sG3jF5fF5cL0rR0kC8lX4oA7yM8gC5rC4hB5'}

In [ ]:
dsfilename = 'fire.pickle'

with open(dsfilename, 'wb') as file:
    pickle.dump(x_train, file)

In [ ]:
url = 'https://api.ibm.com/edgeai/run/modelmgmt/generate_fingerprint' # your url may differ. change as neccessary

data = {'model_type': 'keras', 'partial_activations': 0.2}

multiple_files = [
    ('dataset', ('fire.pickle', open(dsfilename, 'rb'), 'file/pickle')),
    ('model', ('fire.h5', open(model_save_path, 'rb'), 'file/h5'))]

r = requests.post(url, data=data, files=multiple_files, headers=headers, verify=False)

In [ ]:
r

In [ ]:
r.text

In [ ]:
r.json()
task_id = r.json()['task_id']

In [ ]:
url = 'https://api.ibm.com/edgeai/run/modelmgmt/status/{}'.format(task_id)

r = requests.get(url, headers=headers, verify=False)
r.json()  